# SQLite with sqlalchemy
Unfortunately this is an overkill for the current project, but it is a good practice to use it. (in general)

In [5]:
# import numpy as np
# from sqlalchemy import Column, String, Integer, REAL, create_engine
# from sqlalchemy.orm import declarative_base
# from sqlalchemy.orm import sessionmaker
import os

For the SQL Database we need the following:

An object that has all the properties associated to a pairwise prediction

    A) A unique identifier. Will be a hex string chosen from 10^10 numbers there will be a unocupied numbers file.
    B) Path to folder
    C) Both – gene names and gene identifiers, sequences, canonicality, Chain length
    D) Predicted Values – pTM, ipTM, pDockQ, mpDockQ

In [33]:
Base = declarative_base()


class PairwisePrediction(Base):
    __tablename__ = 'AF2 full-lib'

    pid = Column("pid", String, primary_key=True)
    data_path = Column("data_path", String)
    geneA = Column("gene A", String)
    geneB = Column("gene B", String)
    nameA = Column("name A", String)
    nameB = Column("name B", String)
    seqA = Column("seq A", String)
    seqB = Column("seq B", String)
    canonicA = Column("canonic A", Integer)
    canonicB = Column("canonic B", Integer)
    lenA = Column("len A", Integer)
    lenB = Column("len B", Integer)
    pTM = Column("pTM", REAL)
    ipTM = Column("ipTM", REAL)
    pDockQ = Column("pDockQ", REAL)
    meanpDockQ = Column("meanpDockQ", REAL)

    def __init__(self, pid, data_path, geneA, geneB, nameA, nameB, seqA, seqB, canonicA, canonicB, lenA, lenB, pTM,
                 ipTM, pDockQ, meanpDockQ, **kw):
        super().__init__(**kw)
        self.pid = pid
        self.meanpDockQ = meanpDockQ
        self.pDockQ = pDockQ
        self.ipTM = ipTM
        self.pTM = pTM
        self.lenB = lenB
        self.lenA = lenA
        self.canonicB = canonicB
        self.canonicA = canonicA
        self.seqB = seqB
        self.seqA = seqA
        self.nameB = nameB
        self.nameA = nameA
        self.geneB = geneB
        self.data_path = data_path
        self.geneA = geneA

    def __repr__(self):
        return f"{self.__tablename__}({self.pid}, {self.data_path}, {self.geneA}, {self.geneB}, {self.nameA}, {self.nameB}, {self.seqA}, {self.seqB}, {self.canonicA}, {self.canonicB}, {self.lenA}, {self.lenB}, {self.pTM}, {self.ipTM}, {self.pDockQ}, {self.meanpDockQ})"


# %%
engine = create_engine('sqlite:///PPIDB_full.db', echo=True)
Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
session = Session()

interaction = PairwisePrediction(
    pid=str(hex(np.random.randint(10 ** 10))),
    data_path='path/to/data',
    geneA='geneA',
    geneB='geneB',
    nameA='nameA',
    nameB='nameB',
    seqA='seqA',
    seqB='seqB',
    canonicA=1,
    canonicB=1,
    lenA=100,
    lenB=100,
    pTM=0.5,
    ipTM=0.5,
    pDockQ=0.5,
    meanpDockQ=0.5)

print(interaction)

# %%
# session.add(interaction)
# session.commit()

# Pickle file 

In [1]:
import pickle
import os
import pandas as pd

import filecmp


## Step 1 – compare directories and identify the files that are not in the database

In [2]:
# def identity_diff(base_path, previous_data, current_data):
#     changed_items = {'added': [], 'removed': [], 'modified_files': {}}
#     
#     # Compare current data with previous data
#     current_paths = set(current_data.keys())
#     previous_paths = set(previous_data.keys())
# 
#     # Identify added and removed paths
#     added_paths = current_paths - previous_paths
#     removed_paths = previous_paths - current_paths
# 
#     for path_local in added_paths:
#         changed_items['added'].append(path_local)
#     for path_local in removed_paths:
#         changed_items['removed'].append(path_local)
# 
#     # Check for modifications in existing paths
#     for path_local in current_paths.intersection(previous_paths):
#         current_info = current_data[path_local]
#         previous_info = previous_data[path_local]
#         if current_info != previous_info:
#             changed_items[path_local] = current_info
#             modified_files = []
#             for file in current_info['files']:
#                 if file not in previous_info['files'] or \
#                     os.path.getmtime(os.path.join(base_path, path_local, file)) != previous_info['last_modified']:
#                     modified_files.append(file)
#             if modified_files:
#                 changed_items['modified_files'][path_local] = modified_files
#     
#     return changed_items


In [3]:
# def analyze_db(dir_path):
#     # Check if previous data exists
#     previous_data = {}
#     if os.path.exists(os.path.join(dir_path ,'dir_data.pickle')):
#         with open(os.path.join(dir_path ,'dir_data.pickle'), 'rb') as f:
#             previous_data = pickle.load(f)
#     
#     # Get current directory structure
#     current_data = {}
#     for root, dirs, files in os.walk(dir_path):
#         relative_path = os.path.relpath(root, dir_path)
#         current_data[relative_path] = {
#             'dirs': dirs,
#             'files': files,
#             'last_modified': os.path.getmtime(root)
#         }
#     
#     # Identify differences
#     changes = identity_diff(dir_path, previous_data, current_data)
#     
#     # Clean MSA folder from changes for the remvoed folder/
#     changes['removed'] = [x for x in changes['removed'] if 'msas' not in x]
#     
#     # Clean MSA folder from changes for the added folder
#     changes['added'] = [x for x in changes['added'] if 'msas' not in x]
#     # Save current data for future comparison
#     with open(os.path.join(dir_path ,'dir_data.pickle'), 'wb') as f:
#         pickle.dump(current_data, f)
#     return changes
# 
# 
# chgs = analyze_db('/Volumes/dave/JW/PPIDB/full_lib')



In [67]:
# the_path = '/Volumes/dave/JW/PPIDB/full_lib/control_db'
# list_exp_path = [os.path.join(the_path, exp_pth) for exp_pth in os.listdir(the_path) if
#                      os.path.isdir(os.path.join(the_path, exp_pth)) and not exp_pth.startswith('.')]

## Small scale test

Removed paths have to be treated differently, as they are not present in the database.

In [1]:
from aflib.decipher import load_library_aslist, uniquify
import sqlite3
import os
import pickle
import pandas as pd


In [2]:
def identity_diff(base_path, previous_data, current_data):
    changed_items = {'added': [], 'removed': [], 'modified_files': {}}
    
    # Compare current data with previous data
    current_paths = set(current_data.keys())
    previous_paths = set(previous_data.keys())

    # Identify added and removed paths
    added_paths = current_paths - previous_paths
    removed_paths = previous_paths - current_paths

    for path_local in added_paths:
        changed_items['added'].append(path_local)
    for path_local in removed_paths:
        changed_items['removed'].append(path_local)

    # Check for modifications in existing paths
    for path_local in current_paths.intersection(previous_paths):
        current_info = current_data[path_local]
        previous_info = previous_data[path_local]
        if current_info != previous_info:
            changed_items[path_local] = current_info
            modified_files = []
            for file in current_info['files']:
                if file not in previous_info['files'] or \
                    os.path.getmtime(os.path.join(base_path, path_local, file)) != previous_info['last_modified']:
                    modified_files.append(file)
            if modified_files:
                changed_items['modified_files'][path_local] = modified_files
    
    return changed_items

In [3]:
def analyze_db(dir_path):
    # Check if previous data exists
    previous_data = {}
    if os.path.exists(os.path.join(dir_path ,'dir_data.pickle')):
        with open(os.path.join(dir_path ,'dir_data.pickle'), 'rb') as f:
            previous_data = pickle.load(f)
    
    # Get current directory structure
    current_data = {}
    for root, dirs, files in os.walk(dir_path):
        relative_path = os.path.relpath(root, dir_path)
        current_data[relative_path] = {
            'dirs': dirs,
            'files': files,
            'last_modified': os.path.getmtime(root)
        }
    
    # Identify differences
    changes = identity_diff(dir_path, previous_data, current_data)
    
    # Clean MSA folder from changes for the remvoed folder/
    changes['removed'] = [x for x in changes['removed'] if 'msas' not in x]
    
    # Clean MSA folder from changes for the added folder
    changes['added'] = [x for x in changes['added'] if 'msas' not in x]
    # Save current data for future comparison
    with open(os.path.join(dir_path ,'dir_data.pickle'), 'wb') as f:
        pickle.dump(current_data, f)
    return changes


In [4]:
naming_conventions = """Naming conventions for library folders:
1. No spaces in the folder names
2. Folders that are not predictions are prohibited from using '-' in the folder name
"""
print(naming_conventions)

Naming conventions for library folders:
1. No spaces in the folder names
2. Folders that are not predictions are prohibited from using '-' in the folder name



In [5]:
def get_added_df(changes, library_path):
    # Generate a list f paths to reload
    reloadable_paths_raw = changes['added'] + list(changes['modified_files'].keys())
    reloadable_paths = [os.path.join(library_path, path) for path in reloadable_paths_raw if '-' in path]
    
    if not reloadable_paths:
        return pd.DataFrame()
    # Load the library
    res = load_library_aslist(reloadable_paths)
    unique_df = uniquify(pd.DataFrame(res))
    # Remove rows with missing values
    unique_df.dropna(subset=['Mean pDockQ', 'pTM', 'ipTM'], inplace=True, ignore_index=True)
    # Remove the path prefix
    unique_df['Relative Directory'] = unique_df['Directory'].apply(lambda x: x.replace(library_path + os.path.sep, ''))
    
    return unique_df

## Update the Database
The library update is a multistep process:
1. Identify the changes in the library and store them in a dictionary
2. Load the new and modified folders entries into a unique_df dataframe
3. Edit the database
    1. Load Database as a dataframe
    2. Remove the entries that are in the removed directory
    3. Add the new entries
4. Update Database with modified entries

In [93]:
def update_db(library_path):
    
    # Identify the changes in the database
    changes = analyze_db(library_path)
    df_toadd = get_added_df(changes, library_path)
    
    # Load current database
    if not os.path.exists(os.path.join(library_path, 'PPIDB_full.db')):
        print('Database not found – creating a new one')
        cnx = sqlite3.connect(os.path.join(library_path, 'PPIDB_full.db'))
        
    try:
        df_in = pd.read_sql('AF2 full-lib', f"sqlite:///{os.path.join(library_path, 'PPIDB_full.db')}")
        print(df_in['Job name'].head())
        
        # Remove the entries that were removed from the database
        for rem_path in changes['removed']:
            components = os.path.basename(rem_path).split('-')
            # Show row where 'Job name' contains both components
            index = df_in[df_in['Job name'].str.contains(components[0]) & df_in['Job name'].str.contains(components[1])].index
            df_in.drop(index, inplace=True)
    except Exception:
        print('Database not found – considering it do be empty')
        df_in = pd.DataFrame()
    
    # Add the new entries
    df = pd.concat([df_in, df_toadd], ignore_index=True)
    print(df['Job name'].head())
    
    # Save and replace current database with updated one
    cnx = sqlite3.connect(os.path.join(library_path, 'PPIDB_full.db'))
    df.to_sql(name='AF2 full-lib', con=cnx, if_exists='replace', index=False)

In [97]:
update_db('/Volumes/dave/JW/PPIDB/full_lib/control_db')

0        dnd1-lifeact
1      nanos3-lifeact
2       lifeact-actb1
3    LAscramble-actb1
4        dnd1-eif3d_4
Name: Job name, dtype: object
0        dnd1-lifeact
1      nanos3-lifeact
2       lifeact-actb1
3    LAscramble-actb1
Name: Job name, dtype: object
